### Cortez Tomás

## Abalone is marine snails. 
### This data set is about predicting abalone age from physical measurements. The age of abalone is determined by cutting the shell through the cone, staining it, and counting the number of rings through a microscope - a boring and time-consuming task. Other measurements, which are easier to obtain, may be used to predict the age. The data set contains the attributes that report on sex, size and weight measurements. The age is reported as the number of rings, which increased by 1.5 gives the age in years.

### El objetivo es pronosticar la cantidad de anillos (variable “Rings”) en función del resto de las features provistas y generar el modelo listo para Producción.


### Importo las librerias necesarias 

In [255]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import neighbors
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
import plotly.graph_objs as go
import scipy.stats as sts
import plotly.express as px
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

In [456]:
df=pd.read_csv('abalone1_3677.csv')
df_copy = df.copy() #Genero una copia por las dudas
df_copy.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,1.0,0.415,0.325,0.115,0.3285,0.1405,0.0510,0.106,12.0
1,0.0,0.610,0.470,0.145,1.1530,0.4030,0.2960,0.320,14.0
2,1.0,0.190,0.145,0.040,0.0380,0.0165,0.0065,0.015,4.0
3,1.0,0.550,0.420,0.135,0.8160,0.3995,0.1485,0.230,12.0
4,2.0,0.485,0.355,0.120,0.5470,0.2150,0.1615,0.140,10.0


In [63]:
df_copy.describe()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
count,3677.000000,3677.000000,3677.000000,3677.000000,3677.000000,3677.000000,3677.000000,3677.000000,3677.000000
mean,1.053576,0.524373,0.408113,0.139633,0.829556,0.360247,0.180669,0.239167,9.939625
std,0.820780,0.119955,0.099231,0.042335,0.493281,0.223983,0.109950,0.139968,3.218579
min,0.000000,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000
25%,0.000000,0.450000,0.350000,0.115000,0.440500,0.185000,0.092000,0.129500,8.000000
50%,1.000000,0.545000,0.425000,0.140000,0.800500,0.336000,0.171000,0.232500,9.000000
75%,2.000000,0.615000,0.480000,0.165000,1.151000,0.503000,0.253500,0.330000,11.000000
max,2.000000,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000


In [64]:
df_copy.info() #no hay nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3677 entries, 0 to 3676
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             3677 non-null   float64
 1   Length          3677 non-null   float64
 2   Diameter        3677 non-null   float64
 3   Height          3677 non-null   float64
 4   Whole weight    3677 non-null   float64
 5   Shucked weight  3677 non-null   float64
 6   Viscera weight  3677 non-null   float64
 7   Shell weight    3677 non-null   float64
 8   Rings           3677 non-null   float64
dtypes: float64(9)
memory usage: 258.7 KB


## Con este grafico se puede ver dos outliers que los retiro posteriormente

In [77]:
Height = df_copy[~pd.isnull(df_copy['Height'])]['Height']
qq_x, qq_y = sts.probplot(Height, dist="norm", fit=False)
chart = px.scatter(x=qq_x, y=qq_y, trendline='ols', trendline_color_override='red',width=1000, height=500)
chart

### Genero dummies de "Sex" y elimino columnas que no aportan mucho (esto lo consigo con lasso)

In [457]:
df_copy = df_copy.drop(df_copy[df_copy['Height']>=0.4].index)
df_copy = pd.get_dummies(df_copy,columns=['Sex']) #creo las varibles tontas (Sex 1,2,3)
#X = df_copy.drop(["Rings"], axis=1)
X = df_copy.drop(["Rings","Sex_0.0","Length"], axis=1)
Y = df_copy["Rings"]
df_copy

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,Sex_0.0,Sex_1.0,Sex_2.0
0,0.415,0.325,0.115,0.3285,0.1405,0.0510,0.106,12.0,0,1,0
1,0.610,0.470,0.145,1.1530,0.4030,0.2960,0.320,14.0,1,0,0
2,0.190,0.145,0.040,0.0380,0.0165,0.0065,0.015,4.0,0,1,0
3,0.550,0.420,0.135,0.8160,0.3995,0.1485,0.230,12.0,0,1,0
4,0.485,0.355,0.120,0.5470,0.2150,0.1615,0.140,10.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
3672,0.490,0.400,0.115,0.5690,0.2560,0.1325,0.145,9.0,1,0,0
3673,0.670,0.550,0.190,1.3905,0.5425,0.3035,0.400,12.0,1,0,0
3674,0.510,0.395,0.125,0.5805,0.2440,0.1335,0.188,11.0,0,0,1
3675,0.575,0.465,0.120,1.0535,0.5160,0.2185,0.235,9.0,0,0,1


In [458]:
#divido en train y test 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 100)
#divido en train_validation y test_validation
X_val_train, X_val_test, Y_val_train, Y_val_test = train_test_split(X_train,Y_train,test_size=0.2,random_state=100)

## "Normalizadores"

In [478]:
roboust = RobustScaler()
X_val_train_roboust = roboust.fit_transform(X_val_train)
X_val_test_roboust = roboust.transform(X_val_test)

In [479]:
MinMax = MinMaxScaler()
X_val_train_MinMax = MinMax.fit_transform(X_val_train)
X_val_test_MinMax = MinMax.transform(X_val_test)

In [481]:
Standard = StandardScaler()
X_val_train_Standard = Standard.fit_transform(X_val_train)
X_val_test_Standard = Standard.transform(X_val_test)

# modelos
## knn

In [473]:
knn_pru = neighbors.KNeighborsClassifier(n_neighbors=12,weights='distance',p=2)
knn_pru.fit(X_val_train_roboust,Y_val_train) 
knn_pred = knn_pru.predict(X_val_test_roboust)

knn_prul = mean_squared_error(Y_val_test, knn_pred ,squared=False)
print("Evaluación modelo kNN con RobustScaler")
print("RMSE 12 vecinos:",round(knn_prul,3))

Evaluación modelo kNN con RobustScaler
RMSE 12 vecinos: 2.258


In [506]:
knn_MinMax = neighbors.KNeighborsClassifier(n_neighbors=12,weights='distance',p=2)
knn_MinMax.fit(X_val_train_MinMax,Y_val_train) 
knn_pred_MinMax = knn_MinMax.predict(X_val_test_MinMax)

knn_MinMax1 = mean_squared_error(Y_val_test, knn_pred_MinMax ,squared=False)
print("Evaluación modelo kNN con MinMax")
print("RMSE 12 vecinos:",round(knn_MinMax1,3))

Evaluación modelo kNN con MinMax
RMSE 12 vecinos: 2.245


In [505]:
knn_Standard = neighbors.KNeighborsClassifier(n_neighbors=12,weights='distance',p=2)
knn_Standard.fit(X_val_train_Standard,Y_val_train) 
knn_pred_Standard = knn_Standard.predict(X_val_test_Standard)

knn_Standard1 = mean_squared_error(Y_val_test, knn_pred_Standard ,squared=False)
print("Evaluación modelo kNN con Standard")
print("RMSE 12 vecinos:",round(knn_Standard1,3))

Evaluación modelo kNN con Standard
RMSE 12 vecinos: 2.298


In [398]:
knn_pruro = neighbors.KNeighborsClassifier(n_neighbors=8,weights='distance',p=2)
knn_pruro.fit(X_val_train,Y_val_train) 
knn_predro = knn_pruro.predict(X_val_test)

knn_prulro = mean_squared_error(Y_val_test, knn_predro ,squared=False)
print("Evaluación modelo kNN")
print("RMSE 12 vecinos:",round(knn_prulro,3))

Evaluación modelo kNN
RMSE 12 vecinos: 2.188


## arbol

In [424]:
modelo_arbol_ro = DecisionTreeClassifier(max_depth=6, random_state=100)       
modelo_arbol_ro.fit(X_val_train_roboust, Y_val_train)              
arbol_pre_ro = modelo_arbol_ro.predict(X_val_test_roboust)


RMSE_tree_ro=mean_squared_error(Y_val_test, arbol_pre_ro,squared=False)
print("Evaluación del modelo Seleccionado: ")
print("RMSE con RobustScaler y 6 ramas:",round(RMSE_tree_ro,3))

Evaluación del modelo Seleccionado: 
RMSE con RobustScaler y 6 ramas: 2.28


In [513]:
modelo_arbol_Standard = DecisionTreeClassifier(max_depth=6, random_state=100)       
modelo_arbol_Standard.fit(X_val_train_Standard, Y_val_train)              
arbol_pre_Standard = modelo_arbol_Standard.predict(X_val_test_Standard)


RMSE_tree_Standard=mean_squared_error(Y_val_test, arbol_pre_Standard,squared=False)
print("Evaluación del modelo Seleccionado: ")
print("RMSE con Standard y 6 ramas:",round(RMSE_tree_Standard,3))

Evaluación del modelo Seleccionado: 
RMSE con Standard y 6 ramas: 2.258


In [429]:
modelo_arbol = DecisionTreeClassifier(max_depth=6, random_state=100)       
modelo_arbol.fit(X_val_train, Y_val_train)              
Y_pred_tree = modelo_arbol.predict(X_val_test)

RMSE_tree=mean_squared_error(Y_val_test, Y_pred_tree,squared=False)
print("Evaluación del modelo Seleccionado: ")
print("RMSE con 6 ramas:",round(RMSE_tree,3))

#Al modelo de arbol no le importa nada.

Evaluación del modelo Seleccionado: 
RMSE con 6 ramas: 2.277


## svm kernel lineal

In [401]:
SVM_lineal = LinearSVC(max_iter=10000)
SVM_lineal.fit(X_val_train,Y_val_train)
ACC_SVM_lineal=SVM_lineal.score(X_val_test,Y_val_test)
print("Accuracy para SVM Lineal: ", ACC_SVM_lineal)

y_pred_svm = SVM_lineal.predict(X_val_test)

RMSE_svm=mean_squared_error(Y_val_test, y_pred_svm,squared=False)
print("RMSE: ",round(RMSE_svm,3))

Accuracy para SVM Lineal:  0.2534013605442177
RMSE:  2.506


## svm con Kernel no lineal "rbf"

In [560]:
SVM_rbf=SVC(kernel="rbf")
SVM_rbf.fit(X_val_train,Y_val_train)
ACC_rbf=SVM_rbf.score(X_val_test,Y_val_test)
print("Accuracy para SVM con kernel rbf: ", ACC_rbf)

y_pred_svm_k = SVM_rbf.predict(X_val_test)

RMSE_svm_k=mean_squared_error(Y_val_test, y_pred_svm_k,squared=False)
print("RMSE: ",round(RMSE_svm_k,3))

Accuracy para SVM con kernel rbf:  0.24829931972789115
RMSE:  2.787


## svm con kernel polinómico de grado 3

In [561]:
SVM_poly=SVC(kernel="poly", degree=3)
SVM_poly.fit(X_val_train,Y_val_train)
ACC_poly=SVM_poly.score(X_val_test,Y_val_test)
print("Accuracy para SVM con kernel grado 3: ", ACC_poly)

y_pred_svm_k3 = SVM_poly.predict(X_val_test)

RMSE_svm_3=mean_squared_error(Y_val_test, y_pred_svm_k3,squared=False)
print("RMSE: ",round(RMSE_svm_3,3))

Accuracy para SVM con kernel grado 3:  0.24829931972789115
RMSE:  2.709


## svm con kernel sigmoide

In [388]:
SVM_sig=SVC(kernel="sigmoid")
SVM_sig.fit(X_val_train,Y_val_train)
ACC_sig=SVM_sig.score(X_val_test,Y_val_test)
print("Accuracy para SVM con kernel sigmoide: ", ACC_sig)
y_pred_sig = SVM_sig.predict(X_val_test)

RMSE_sig=mean_squared_error(Y_val_test, y_pred_sig,squared=False)
print("RMSE: ",round(RMSE_svm,3))

Accuracy para SVM con kernel sigmoide:  0.1717687074829932
RMSE:  2.516


## svm polinómico grado 4

In [330]:
for a in [10,100,1000]:    
    
    svm=SVC(C=a, kernel='poly',degree=4)
    svm.fit(X_val_train,Y_val_train)
    acc=svm.score(X_val_test,Y_val_test)
    
    print("--------------------------------")
    print("Kernel:poly, grado 4")
    print("C: ",a)
    print("Accuracy: ",acc)

--------------------------------
Kernel:poly, grado 4
C:  10
Accuracy:  0.2614601018675722
--------------------------------
Kernel:poly, grado 4
C:  100
Accuracy:  0.2563667232597623
--------------------------------
Kernel:poly, grado 4
C:  1000
Accuracy:  0.2461799660441426


## lasso

In [514]:
modelo_lasso = Lasso(alpha=0.0001)
modelo_lasso.fit(X_val_train,Y_val_train)
#modelo_lasso.score(X_val_test,Y_val_test)
y_pred_lasso = modelo_lasso.predict(X_val_test)

RMSE_lasso=mean_squared_error(Y_val_test, y_pred_lasso,squared=False)
print("RMSE: ",round(RMSE_lasso,3))

RMSE:  2.013


In [532]:
modelo_lasso_Standard = Lasso(alpha=0.0001)
modelo_lasso_Standard.fit(X_val_train_Standard,Y_val_train)
#modelo_lasso.score(X_val_test,Y_val_test)
y_pred_lasso_Standard = modelo_lasso_Standard.predict(X_val_test_Standard)

RMSE_lasso_Standard=mean_squared_error(Y_val_test, y_pred_lasso_Standard,squared=False)
print("RMSE: ",round(RMSE_lasso_Standard,3))

RMSE:  2.012


In [344]:
modelo_lasso.coef_ #en teoria las columnas ("Sex=0" y "Length") no me aporta nada por eso la saco mas arriba

array([  1.28897118,   9.86113097,   7.48766853,   7.04688034,
       -17.91262022,  -7.9364442 ,  10.77733991,   0.        ,
        -0.78183498,   0.09542269])

In [404]:
Log_Lasso=LogisticRegression(penalty='l1', solver='saga',C=1, max_iter=5000, multi_class='ovr') 
Log_Lasso.fit(X_val_train,Y_val_train)
Log_Lasso.score(X_val_test,Y_val_test)

0.25

## Ridge

In [567]:
modelo_ridge=Ridge(alpha=0.01)
modelo_ridge.fit(X_val_train_roboust,Y_val_train)
y_pred_Ridge = modelo_ridge.predict(X_val_test_roboust)

RMSE_Ridge=mean_squared_error(Y_val_test, y_pred_Ridge,squared=False)
print("RMSE_Ridge: ",round(RMSE_Ridge,3))

RMSE_Ridge:  2.012


In [565]:
modelo_ridge_Standard=Ridge(alpha=1)
modelo_ridge_Standard.fit(X_val_train_Standard,Y_val_train)
y_pred_Ridge_Standard = modelo_ridge_Standard.predict(X_val_test_Standard)

RMSE_Ridge_Standard=mean_squared_error(Y_val_test, y_pred_Ridge_Standard,squared=False)
print("RMSE_Ridge: ",round(RMSE_Ridge_Standard,3))

RMSE_Ridge:  2.012


In [533]:
Log_Ridge=LogisticRegression(penalty='l2', C=125, max_iter=1000, multi_class='ovr')
Log_Ridge.fit(X_val_train,Y_val_train)
Log_Ridge.score(X_val_test,Y_val_test)
y_pred_Ridge1 = Log_Ridge.predict(X_val_test)

RMSE_Ridge1=mean_squared_error(Y_val_test, y_pred_Ridge1,squared=False)
print("RMSE_Ridge: ",round(RMSE_Ridge1,3))

RMSE_Ridge:  2.41


## ElasticNet

In [547]:
enet = ElasticNet(alpha=0.0001, l1_ratio=0.7,)
enet.fit(X_val_train, Y_val_train)
y_pred_enet = enet.predict(X_val_test)

RMSE_enet=mean_squared_error(Y_val_test, y_pred_enet,squared=False)
print("RMSE_Ridge: ",round(RMSE_enet,3))

RMSE_Ridge:  2.016


In [590]:
enet_Standard = ElasticNet(alpha=0.0001, l1_ratio=0.5)
enet_Standard.fit(X_val_train_Standard, Y_val_train)
y_pred_enet_Standard = enet_Standard.predict(X_val_test_Standard)

RMSE_enet_Standard=mean_squared_error(Y_val_test, y_pred_enet_Standard,squared=False)
print("RMSE_Ridge: ",round(RMSE_enet_Standard,3))

RMSE_Ridge:  2.012


## Standard Para train y test

In [571]:
Standard = StandardScaler()
X_train_Standard = Standard.fit_transform(X_train)
X_test_Standard = Standard.transform(X_test)

## arbol estandarizado

In [552]:
modelo_arbol_Standard_final = DecisionTreeClassifier(max_depth=6, random_state=100)       
modelo_arbol_Standard_final.fit(X_train_Standard, Y_train)              
arbol_pre_Standard_final = modelo_arbol_Standard_final.predict(X_test_Standard)


RMSE_tree_Standard_final=mean_squared_error(Y_test, arbol_pre_Standard_final,squared=False)
print("Evaluación del modelo Seleccionado: ")
print("RMSE con Standard y 6 ramas:",round(RMSE_tree_Standard_final,3))

Evaluación del modelo Seleccionado: 
RMSE con Standard y 6 ramas: 2.604


## kkn puro

In [559]:
knn_puro = neighbors.KNeighborsClassifier(n_neighbors=8,weights='distance',p=2)
knn_puro.fit(X_train,Y_train) 
knn_y_puro = knn_puro.predict(X_test)

knn_puro1 = mean_squared_error(Y_test, knn_y_puro ,squared=False)
print("Evaluación modelo kNN")
print("RMSE 12 vecinos:",round(knn_puro1,3))

Evaluación modelo kNN
RMSE 12 vecinos: 2.585


## svm kernel lineal

In [563]:
SVM_lineal_final = LinearSVC(max_iter=10000)
SVM_lineal_final.fit(X_train,Y_train)
y_pred_svm_final = SVM_lineal_final.predict(X_test)

RMSE_svm_final=mean_squared_error(Y_test, y_pred_svm_final,squared=False)
print("RMSE: ",round(RMSE_svm_final,3))

RMSE:  2.632


## Ridge Standard

In [568]:
modelo_ridge_final_Standard=Ridge(alpha=0.01)
modelo_ridge_final_Standard.fit(X_train_Standard,Y_train)
y_pred_Ridge_final = modelo_ridge_final_Standard.predict(X_test_Standard)

RMSE_Ridge_final=mean_squared_error(Y_test, y_pred_Ridge_final,squared=False)
print("RMSE_Ridge: ",round(RMSE_Ridge_final,3))

RMSE_Ridge:  2.279


## lasso Standard

In [569]:
modelo_lasso_Standard_final = Lasso(alpha=0.0001)
modelo_lasso_Standard_final.fit(X_train_Standard,Y_train)
#modelo_lasso.score(X_val_test,Y_val_test)
y_pred_lasso_Standard_final = modelo_lasso_Standard_final.predict(X_test_Standard)

RMSE_lasso_Standard_final=mean_squared_error(Y_test, y_pred_lasso_Standard_final,squared=False)
print("RMSE: ",round(RMSE_lasso_Standard_final,3))

RMSE:  2.279


## ElasticNet

In [591]:
enet_Standard_final = ElasticNet(alpha=0.0001, l1_ratio=0.5)
enet_Standard_final.fit(X_train_Standard, Y_train)
y_pred_enet_Standard_final = enet_Standard_final.predict(X_test_Standard)

RMSE_enet_Standard_final=mean_squared_error(Y_test, y_pred_enet_Standard_final,squared=False)
print("RMSE_Ridge: ",round(RMSE_enet_Standard_final,3))

RMSE_Ridge:  2.278


## Modelo final

In [ ]:
#X = df_copy.drop(["Rings","Sex_0.0","Length"], axis=1)
#Y = df_copy["Rings"]
#para acordarme

In [594]:
Standard = StandardScaler()
X_Standard = Standard.fit_transform(X)

In [595]:
modelo_prod = ElasticNet(alpha=0.0001, l1_ratio=0.5)
modelo_prod.fit(X_Standard, Y)

ElasticNet(alpha=0.0001)

In [596]:
y_pred_enet_Standard_final1 = modelo_prod.predict(X_Standard)

RMSE_enet_Standard_final1=mean_squared_error(Y, y_pred_enet_Standard_final1,squared=False)
print("RMSE_Ridge: ",round(RMSE_enet_Standard_final1,3))

RMSE_Ridge:  2.183


## importo el df para pronosticar

In [602]:
df2=pd.read_csv('abalone_500_X.csv')
df2

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight
0,2.0,0.605,0.455,0.160,1.1035,0.4210,0.3015,0.325
1,2.0,0.590,0.440,0.150,0.8725,0.3870,0.2150,0.245
2,0.0,0.560,0.445,0.195,0.9810,0.3050,0.2245,0.335
3,0.0,0.635,0.490,0.170,1.2615,0.5385,0.2665,0.380
4,2.0,0.475,0.385,0.145,0.6175,0.2350,0.1080,0.215
...,...,...,...,...,...,...,...,...
495,2.0,0.550,0.460,0.130,0.7085,0.3050,0.1455,0.205
496,0.0,0.605,0.495,0.170,1.2385,0.5280,0.2465,0.390
497,0.0,0.600,0.485,0.165,1.1405,0.5870,0.2175,0.288
498,0.0,0.330,0.260,0.080,0.2000,0.0625,0.0500,0.070


## replico los pasos par aplicar el modelo

In [603]:
df2 = df2.drop(df2[df2['Height']>=0.4].index)
df2 = pd.get_dummies(df2,columns=['Sex']) #creo las varibles tontas (Sex 1,2,3)
#X = df_copy.drop(["Rings"], axis=1)
df2 = df2.drop(["Sex_0.0","Length"], axis=1)
df2

,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Sex_1.0,Sex_2.0
0,0.455,0.160,1.1035,0.4210,0.3015,0.325,0,1
1,0.440,0.150,0.8725,0.3870,0.2150,0.245,0,1
2,0.445,0.195,0.9810,0.3050,0.2245,0.335,0,0
3,0.490,0.170,1.2615,0.5385,0.2665,0.380,0,0
4,0.385,0.145,0.6175,0.2350,0.1080,0.215,0,1
...,...,...,...,...,...,...,...,...
495,0.460,0.130,0.7085,0.3050,0.1455,0.205,0,1
496,0.495,0.170,1.2385,0.5280,0.2465,0.390,0,0
497,0.485,0.165,1.1405,0.5870,0.2175,0.288,0,0
498,0.260,0.080,0.2000,0.0625,0.0500,0.070,0,0


## hago la prediccion

In [606]:
rings_predict=modelo_prod.predict(df2)
len(rings_predict)

500

In [607]:
rings_predict_csv = pd.DataFrame(rings_predict,columns=['Rings'])
rings_predict_csv.to_csv('Cortez_Tomas.csv',index=False)

In [ ]:
#     :)